In [1]:
import pymongo
import json
from pyspark.sql import *

In [2]:
DATA_PATH = '../data/college-search-data-v3.parquet'
IMAGE_DATA_FILE = './college_picture_urls.json'

In [3]:
uri = "mongodb://cledge-db:Tc7PYDfYtB1jBrPfBDPYSNLhlovMKU6AatyKbWPAE75IFy1tLEsczdULMYfaoiFu5bH9qbna704PQCdHBKT8YQ==@cledge-db.mongo.cosmos.azure.com:10255/?ssl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@cledge-db@" # in cledge-db resource, then click Quick Start.
client = pymongo.MongoClient(uri)

In [4]:
db = client["colleges"]
collection = db["colleges"]

## Set up data that is needed to transfer to Azure Cosmos DB

In [5]:
# Initialize SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [6]:
# Load data
df = spark.read.load(DATA_PATH)

In [7]:
# column needed for college list
columns = ["UNITID", "INSTNM", "TARGET_TIER", "SAFETY_TIER", "CITY", "STABBR", "TUITIONFEE_IN", "TUITIONFEE_OUT", "CONTROL"]

In [8]:
with open(IMAGE_DATA_FILE) as f:
    pic_data = json.load(f)

In [9]:
for row in df.select(columns).collect():
    curLocation = None
    if row.CITY is not None and row.STABBR is not None:
        curLocation = row.CITY + ", " + row.STABBR
    curCollegeType = None
    if row.CONTROL is not None:
        if row.CONTROL == 1: curCollegeType = "Public"
        elif row.CONTROL == 2: curCollegeType = "Private non-profit"
        elif row.CONTROL == 3: curCollegeType = "Private for-profit"
    curCollege = {
        "college_id": row.UNITID,
        "img_url": pic_data[row.UNITID]["img_link"],
        "img_title": pic_data[row.UNITID]["img_title"],
        "college_name": row.INSTNM,
        "target_tier": row.TARGET_TIER,
        "safety_tier": row.SAFETY_TIER,
        "location": curLocation,
        "in_state_tuition": row.TUITIONFEE_IN,
        "out_state_tuition": row.TUITIONFEE_OUT,
        "college_type": curCollegeType
    }
    collection.insert_one(curCollege)

In [11]:
result = collection.find({"college_id": "100654"})
result[0]

{'_id': ObjectId('62dba5a89e458c45472fc2fc'),
 'college_id': '100654',
 'img_url': 'https://upload.wikimedia.org/wikipedia/commons/d/d8/AAMU_James_H._Wilson_Hall_Dec10_01.jpg',
 'img_title': '',
 'college_name': 'Alabama A & M University',
 'target_tier': 0,
 'safety_tier': 3,
 'location': 'Normal, AL',
 'in_state_tuition': 10024,
 'out_state_tuition': 18634,
 'college_type': 'Public'}